In [4]:
import re, time, requests, json
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
okt = Okt()

with open('C:/Users/twentystones/Desktop/사회연결망분석텀프로젝트/영석 수정중/KnuSentiLex-master/KnuSentiLex/data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)

dict_knu = pd.DataFrame(data)
dict_knu

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14838,반신반의하다,반신반의,0
14839,신비롭다,신비,1
14840,아리송하다,아리송,-1
14841,알쏭하다,알쏭하,-1


In [208]:
city_list = ['bangkok','cebu','danang','fukuoka','hanoi','hongkong','osaka','sapporo','taipei','tokyo']

#지역별 긍부정단어 찾기
score_word = pd.DataFrame(columns = ['score_two_word', 'score_one_word', 'score_mone_word', 'score_mtwo_word'] )

for city in city_list:     ##################################
    print(city)    ######
    
    #긍부정단어 초기화
    score_two_word, score_one_word, score_mone_word, score_mtwo_word = [],[],[],[]

    #데이터전처리
    df = pd.read_csv('C:/Users/twentystones/Desktop/사회연결망분석텀프로젝트/travel_csv/'+city+'_blog_crawling.csv', encoding='utf-8-sig', header=None)
    df.rename(columns= {0:'title', 1:'URL', 2:'day', 3:'body'}, inplace=True)
    df = df.iloc[:,3:]
    df = df.fillna('')
    df

    # 명사 추출
    body_okt_nouns = []
    for i in range(len(df)):    
        body_okt_nouns.append(okt.nouns(df.body[i]))
    df['body_okt_nouns'] = body_okt_nouns 

    #한글자짜리 지우기
    body_okt_nouns_len = []
    for i in range(len(df)):
        body_okt_nouns_len.append([])

    for i in range(len(body_okt_nouns)):
        for word in body_okt_nouns[i]:
            if (len(word)!=1)&(word not in ['살기','환상','가짜','부담']):
                body_okt_nouns_len[i].append(word)
    df['body_okt_nouns'] = body_okt_nouns_len

    #df에 점수넣을 공간 만들기
    df['score_two'], df['score_one'], df['score_mone'], df['score_mtwo'] =  0,0,0,0


    #감성분석
    score_list = []

    for n,i in tqdm(enumerate(df['body_okt_nouns'])):   #한 블로그의 글    ##################
        score_two, score_one, score_zero, score_mone, score_mtwo = 0, 0, 0, 0, 0
        for w in i:   #한 블로그 글 안의 각 단어들

            for k in range(0, len(dict_knu)):

                if dict_knu['word'][k] == w:

                    if int(dict_knu['polarity'][k]) == 2:
                        score_two_word.append(w)
                        score_two += 1

                    elif int(dict_knu['polarity'][k]) == 1:
                        score_one_word.append(w)
                        score_one += 1

                    elif int(dict_knu['polarity'][k]) == -1:
                        score_mone_word.append(w)
                        score_mone += 1

                    else:
                        score_mtwo_word.append(w)
                        score_mtwo += 1

        df['score_two'][n], df['score_one'][n], df['score_mone'][n], df['score_mtwo'][n] =  score_two, score_one, score_mone,score_mtwo
    df.to_csv(city+'긍부정.csv', encoding='utf-8-sig')   #한지역 끝날때마다 저장

    score_word = score_word.append({'score_two_word' :score_two_word,
                         'score_one_word' : score_one_word,
                         'score_mone_word' : score_mone_word,
                         'score_mtwo_word' : score_mtwo_word }, ignore_index=True)
    
score_word.to_csv('긍부정단어목록.csv', encoding='utf-8-sig')    #모든지역 끝났을떄 저장

bangkok


0it [00:00, ?it/s]<ipython-input-208-5bd31a90b6e6>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_two'][n], df['score_one'][n], df['score_mone'][n], df['score_mtwo'][n] =  score_two, score_one, score_mone,score_mtwo
90it [34:43, 23.15s/it]


cebu


90it [25:38, 17.10s/it]


danang


...

90it [31:22, 20.92s/it]


taipei


90it [33:38, 22.43s/it]


tokyo


90it [34:47, 23.20s/it]
